In [42]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [43]:
file_name = "aes128_table_ecb"

In [44]:
nodeset = pd.read_csv(f"../out/features.csv")
df = pd.read_csv(f"../out/edges.csv")

In [45]:
if "label" in nodeset.columns:
    label = "label"
else:
    label = "xor" # suppose xor is the label
# In the paper, nodes belonging to the Sbox and Mixcolumns modules are labeled as 1

X = nodeset.drop(columns=[label])
y = nodeset[label]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

class_values = sorted(nodeset[label].unique())
class_idx = {name: id for id, name in enumerate(class_values)}
paper_idx = {name: idx for idx, name in enumerate(sorted(nodeset["node_number"].unique()))}

nodeset["node_number"] = nodeset["node_number"].apply(lambda name: paper_idx[name])
df["source"] = df["source"].apply(lambda name: paper_idx[name])
df["target"] = df["target"].apply(lambda name: paper_idx[name])
nodeset[label] = nodeset[label].apply(lambda value: class_idx[value])

print(nodeset.xor.value_counts())

xor
0    172
1     55
Name: count, dtype: int64


In [46]:
train_data = nodeset.iloc[0:173]
test_data = nodeset.iloc[173:]

In [47]:
hidden_units = [32, 32]
learning_rate = 0.0001
dropout_rate = 0.3
num_epochs = 32
batch_size = 20

In [48]:
def run_experiment(model, x_train, y_train):
    # Compile the model.
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.BinaryCrossentropy(from_logits=False),
        metrics=[keras.metrics.BinaryAccuracy(name="acc"),
keras.metrics.Precision(), keras.metrics.Recall()],
    )
    # Create an early stopping callback.
    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_acc", patience=50, restore_best_weights=True
    )
    # Fit the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_split=0.10,
        callbacks=[early_stopping],
    )

    return history

In [49]:
def display_learning_curves(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(history.history["loss"])
    ax1.plot(history.history["val_loss"])
    ax1.legend(["train", "test"], loc="upper right")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")

    ax2.plot(history.history["acc"])
    ax2.plot(history.history["val_acc"])
    ax2.legend(["train", "test"], loc="upper right")
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Accuracy")
    plt.show()

In [50]:
def create_ffn(hidden_units, dropout_rate, name=None):
    fnn_layers = []

    for units in hidden_units:
        fnn_layers.append(layers.BatchNormalization())
        fnn_layers.append(layers.Dropout(dropout_rate))
        fnn_layers.append(layers.Dense(units, activation=tf.nn.relu))

    return keras.Sequential(fnn_layers, name=name)

In [51]:
feature_names = set(nodeset.columns) - {"node_number", "Node", "node", label}
num_features = len(feature_names)
num_classes = len(class_idx)

print(train_data[list(feature_names)])
# Create train and test features as a numpy array.
x_train = train_data[list(feature_names)].to_numpy()
x_test = test_data[list(feature_names)].to_numpy()
#print(x_train)
# Create train and test targets as a numpy array.
y_train = train_data[label]
y_test = test_data[label]

     or  and  Degree  mux  Hamming distance  Paths
0     0    0      18    1              3607      0
1     0    0       1    0                 0      0
2     0    0       9    1              8747      1
3     0    0       3    0                 0    510
4     0    0       3    0                 0      0
..   ..  ...     ...  ...               ...    ...
168   0    0       4    0                 0      0
169   0    0       2    0              1016     51
170   0    0       2    0                 0      0
171   0    0       1    0                 0      0
172   0    0       8    0                 0      0

[173 rows x 6 columns]


In [52]:
def create_baseline_model(hidden_units, num_classes, dropout_rate=0.2):
    inputs = layers.Input(shape=(num_features,), name="input_features")
    x = create_ffn(hidden_units, dropout_rate, name=f"ffn_block1")(inputs)
    for block_idx in range(4):
        # Create an FFN block.
        x1 = create_ffn(hidden_units, dropout_rate, name=f"ffn_block{block_idx + 2}")(x)
        # Add skip connection.
        x = layers.Add(name=f"skip_connection{block_idx + 2}")([x, x1])
    # Compute logits.
    logits = layers.Dense(num_classes-1, name="logits")(x)
    # Create the model.
    return keras.Model(inputs=inputs, outputs=logits, name="baseline")


baseline_model = create_baseline_model(hidden_units, num_classes, dropout_rate)
baseline_model.summary()

history = run_experiment(baseline_model, x_train, y_train)

Model: "baseline"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block1          │ (None, 32)        │      1,432 │ input_features[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block2          │ (None, 32)        │      2,368 │ ffn_block1[0][0]  │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection2    │ (None, 32)        │          0 │ ffn_block1[0][0], │
│ (Add)               │                   │            │ ffn_block2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block3          │ (None, 32)        │      2,368 │ skip_connection2… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection3    │ (None, 32)        │          0 │ skip_connection2… │
│ (Add)               │                   │            │ ffn_block3[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block4          │ (None, 32)        │      2,368 │ skip_connection3… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection4    │ (None, 32)        │          0 │ skip_connection3… │
│ (Add)               │                   │            │ ffn_block4[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_block5          │ (None, 32)        │      2,368 │ skip_connection4… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ skip_connection5    │ (None, 32)        │          0 │ skip_connection4… │
│ (Add)               │                   │            │ ffn_block5[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ logits (Dense)      │ (None, 1)         │         33 │ skip_connection5… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,937 (42.72 KB)

 Trainable params: 10,349 (40.43 KB)

 Non-trainable params: 588 (2.30 KB)

Epoch 1/32
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - acc: 0.3186 - loss: 9.5923 - precision: 0.2332 - recall: 0.8082 - val_acc: 0.7778 - val_loss: 2.2790 - val_precision: 0.6000 - val_recall: 0.6000
Epoch 2/32
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.3014 - loss: 10.1331 - precision: 0.2436 - recall: 0.7612 - val_acc: 0.7778 - val_loss: 2.2543 - val_precision: 0.6000 - val_recall: 0.6000
Epoch 3/32
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.3599 - loss: 9.3231 - precision: 0.2660 - recall: 0.9185 - val_acc: 0.6111 - val_loss: 2.2364 - val_precision: 0.3333 - val_recall: 0.4000
Epoch 4/32
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.3570 - loss: 9.3565 - precision: 0.3021 - recall: 0.8076 - val_acc: 0.6667 - val_loss: 2.2376 - val_precision: 0.4000 - val_recall: 0.4000
Epoch 5/32
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc: 0.2954 - loss: 9.4507 - precision: 0.2155 - recall: 0.6757 - val_acc: 0.6111 - val_loss: 2.2988 - val_precision: 0.3750 - val_recall: 0.6000
Epoch 6/32
8/8 ━━━

In [53]:
# Create an edges array (sparse adjacency matrix) of shape [2, num_edges].
edges = df[["source", "target"]].to_numpy().T
edge_weights = tf.ones(shape=edges.shape[1])

node_features = tf.cast(
    nodeset.sort_values("node_number")[list(feature_names)].to_numpy(), dtype=tf.dtypes.float32
)
# Create graph with node features, edges, and edge_weights.
graph_info = (node_features, edges, edge_weights)

print("Edges shape:", edges.shape)
print("Nodes shape:", node_features.shape)
graph_info

Edges shape: (2, 556)
Nodes shape: (227, 6)


(<tf.Tensor: shape=(227, 6), dtype=float32, numpy=
 array([[0.000e+00, 0.000e+00, 1.800e+01, 1.000e+00, 3.607e+03, 0.000e+00],
        [0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [0.000e+00, 0.000e+00, 9.000e+00, 1.000e+00, 8.747e+03, 1.000e+00],
        ...,
        [0.000e+00, 0.000e+00, 2.000e+00, 0.000e+00, 0.000e+00, 3.100e+01],
        [0.000e+00, 0.000e+00, 3.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
        [0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00]],
       dtype=float32)>,
 array([[220,  44, 191, ..., 157,  84, 180],
        [112,   7,  56, ..., 215, 148,  60]]),
 <tf.Tensor: shape=(556,), dtype=float32, numpy=
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.

In [54]:
class GraphConvLayer(layers.Layer):
    def __init__(
        self,
        hidden_units,
        dropout_rate=0.3,
        aggregation_type="mean",
        combination_type="concat",
        normalize=False,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.normalize = normalize

        self.ffn_prepare = create_ffn(hidden_units, dropout_rate)
        if self.combination_type == "gated":
            self.update_fn = layers.GRU(
                units=hidden_units,
                activation="tanh",
                recurrent_activation="sigmoid",
                dropout=dropout_rate,
                return_state=True,
                recurrent_dropout=dropout_rate,
            )
        else:
            self.update_fn = create_ffn(hidden_units, dropout_rate)

    def prepare(self, node_repesentations, weights=None):
        # node_repesentations shape is [num_edges, embedding_dim].
        messages = self.ffn_prepare(node_repesentations)
        if weights is not None:
            messages = messages * tf.expand_dims(weights, -1)
        return messages

    def aggregate(self, node_indices, neighbour_messages, node_repesentations):

        num_nodes = node_repesentations.shape[0]
        if self.aggregation_type == "sum":
            aggregated_message = tf.math.unsorted_segment_sum(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "mean":
            aggregated_message = tf.math.unsorted_segment_mean(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "max":
            aggregated_message = tf.math.unsorted_segment_max(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        else:
            raise ValueError(f"Invalid aggregation type: {self.aggregation_type}.")

        return aggregated_message

    def update(self, node_repesentations, aggregated_messages):

        if self.combination_type == "gru":
            # Create a sequence of two elements for the GRU layer.
            h = tf.stack([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "concat":
            # Concatenate the node_repesentations and aggregated_messages.
            h = tf.concat([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "add":
            # Add node_repesentations and aggregated_messages.
            h = node_repesentations + aggregated_messages
        else:
            raise ValueError(f"Invalid combination type: {self.combination_type}.")

        node_embeddings = self.update_fn(h)
        if self.combination_type == "gru":
            node_embeddings = tf.unstack(node_embeddings, axis=1)[-1]

        if self.normalize:
            node_embeddings = tf.nn.l2_normalize(node_embeddings, axis=-1)
        return node_embeddings

    def call(self, inputs):

        node_repesentations, edges, edge_weights = inputs

        node_indices, neighbour_indices = edges[0], edges[1]
        neighbour_repesentations = tf.gather(node_repesentations, neighbour_indices)
        neighbour_messages = self.prepare(neighbour_repesentations, edge_weights)

        aggregated_messages = self.aggregate(
            node_indices, neighbour_messages, node_repesentations
        )

        return self.update(node_repesentations, aggregated_messages)

In [55]:
class GNNNodeClassifier(tf.keras.Model):
    def __init__(
        self,
        graph_info,
        num_classes,
        hidden_units,
        aggregation_type="sum",
        combination_type="concat",
        dropout_rate=0.3,
        normalize=True,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        # Unpack graph_info to three elements: node_features, edges, and edge_weight.
        node_features, edges, edge_weights = graph_info
        self.node_features = node_features
        self.edges = edges
        self.edge_weights = edge_weights
        # Set edge_weights to ones if not provided.
        if self.edge_weights is None:
            self.edge_weights = tf.ones(shape=edges.shape[1])
        # Scale edge_weights to sum to 1.
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

        # Create a process layer.
        self.preprocess = create_ffn(hidden_units, dropout_rate, name="preprocess")
        # Create the first GraphConv layer.
        self.conv1 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv1",
        )
        # Create the second GraphConv layer.
        self.conv2 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv2",
        )
        # Create a postprocess layer.
        self.postprocess = create_ffn(hidden_units, dropout_rate, name="postprocess")
        # Create a compute logits layer.
        self.compute_logits = layers.Dense(units=num_classes,activation="softmax", name="logits")

    def call(self, input_node_indices):
        # Preprocess the node_features to produce node representations.
        x = self.preprocess(self.node_features)
        # Apply the first graph conv layer.
        x1 = self.conv1((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x1 + x
        # Apply the second graph conv layer.
        x2 = self.conv2((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x2 + x
        # Postprocess node embedding.
        x = self.postprocess(x)
        # Fetch node embeddings for the input node_indices.
        node_embeddings = tf.gather(x, input_node_indices)
        print(node_embeddings)
        # Compute logits
        return self.compute_logits(node_embeddings)

In [58]:
gnn_model = GNNNodeClassifier(
    graph_info=graph_info,
    num_classes=num_classes,
    hidden_units=hidden_units,
    dropout_rate=dropout_rate,
    name="gnn_model",
)

print("GNN output shape:", gnn_model(tf.constant([[1, 10, 100]], dtype=tf.int32)))

gnn_model.summary()

Tensor("GatherV2:0", shape=(1, 3, 32), dtype=float32)
tf.Tensor(
[[[0.16505907 0.06232402 0.         0.         0.20010781 0.37051222
   0.         0.         0.0317753  0.2542474  0.06731512 0.
   0.         0.03398744 0.         0.         0.30601317 0.
   0.01411474 0.2952409  0.         0.03910398 0.         0.
   0.14171825 0.         0.22647277 0.1618072  0.         0.10240357
   0.         0.05749996]
  [1.0301948  0.25048113 0.         0.         1.3852748  1.0172032
   0.         0.         0.         0.07357043 0.21307935 0.
   0.         0.74481833 0.         0.         0.         0.
   0.8963365  0.57160777 0.         1.1142474  0.7879959  0.
   0.         0.         0.4837578  1.0602292  0.         0.14340681
   0.6889345  0.11878217]
  [1.0301948  0.25048113 0.         0.         1.3852748  1.0172032
   0.         0.         0.         0.07357043 0.21307935 0.
   0.         0.74481833 0.         0.         0.         0.
   0.8963365  0.57160777 0.         1.1142474  0.787

Model: "gnn_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ preprocess (Sequential)         │ (227, 32)              │         1,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ graph_conv1 (GraphConvLayer)    │ ?                      │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ graph_conv2 (GraphConvLayer)    │ ?                      │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ postprocess (Sequential)        │ (227, 32)              │         2,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ logits (Dense)                  │ (1, 3, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,642 (61.10 KB)

 Trainable params: 14,798 (57.80 KB)

 Non-trainable params: 844 (3.30 KB)

In [60]:
y_train1 = tf.keras.utils.to_categorical(
    y_train, num_classes=2)
y_test1 = tf.keras.utils.to_categorical(
    y_test, num_classes=2)

In [61]:
x_train = train_data.node_number.to_numpy()
history = run_experiment(gnn_model, x_train, y_train1)

Epoch 1/32
Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
1/8 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - acc: 0.8000 - loss: 16.0877 - precision_1: 0.8000 - recall_1: 0.8000Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - acc: 0.7657 - loss: 54.4738 - precision_1: 0.7657 - recall_1: 0.7657 - val_acc: 0.7222 - val_loss: 38.1303 - val_precision_1: 0.7222 - val_recall_1: 0.7222
Epoch 2/32
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - acc: 0.7549 - loss: 64.8753 - precision_1: 0.7549 - recall_1: 0.7549 - val_acc: 0.7222 - val_loss: 36.0925 - val_precision_1: 0.7222 - val_recall_1: 0.7222
Epoch 3/32
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.7733 - loss: 36.3424 - precision_1: 0.7733 - recall_1: 0.7733 - val_acc: 0.7222 - val_loss: 34.1331 - val_precision_1: 0.7222 - val_recall_1: 0.7222
Epoch 4/32
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - acc: 0.7770 - loss: 38.410

In [67]:
gnn_model.save_weights("../out/saved_weights.weights.h5")

In [73]:
model = GNNNodeClassifier(
    graph_info=graph_info,
    num_classes=num_classes,
    hidden_units=hidden_units,
    dropout_rate=dropout_rate,
    name="gnn_model",
)

model.build(input_shape=(None, graph_info[0].shape[1]))

model.load_weights('../out/saved_weights.weights.h5')

/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'gnn_model', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [74]:
from keras import backend as K
model.compile()
x_test = test_data.node_number.to_numpy()
_, test_accuracy, precision, recall = gnn_model.evaluate(x=x_test, y=y_test1, verbose=0)
print(f"Test accuracy: {round(test_accuracy * 100, 2)}%")
print(f"Test precision: {(precision* 100)}%")
print(f"Test recall: {(recall * 100)}%")

Test accuracy: 77.78%
Test precision: 77.77777910232544%
Test recall: 77.77777910232544%


In [75]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

y_pred= gnn_model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_testt = np.argmax(y_test1, axis=1)
cm = confusion_matrix(y_testt, y_pred)
print(cm)
print(classification_report(y_testt,y_pred))

Tensor("gnn_model_1/GatherV2:0", shape=(32, 32), dtype=float32)
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/stepTensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
[[42  0]
 [12  0]]
              precision    recall  f1-score   support

           0       0.78      1.00      0.88        42
           1       0.00      0.00      0.00        12

    accuracy                           0.78        54
   macro avg       0.39      0.50      0.44        54
weighted avg       0.60      0.78      0.68        54



/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier